# Imports

In [18]:
!pip install youtube_dl
!pip install pytube
!pip install deepspeech
!pip install ffmpeg
!pip install srt

!git clone https://github.com/XniceCraft/ffmpeg-colab.git
!chmod +x ./ffmpeg-colab/install
!./ffmpeg-colab/install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.2 MB 24.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=1cd9245d41b347beada9c1d1723525235b3397e23455ebd04da1169b6f4d89cd
  Stored in directory: /root/.cache/pip/wheels/30/33/46/5ab7eca55b9490dddbf3441c68a29535996270ef1ce8b9b6d7
Successfully built ffmpeg
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for srt: filename=srt-3.5.2-py3-none-any.whl size=22487 sha256=6608ac73c8f6b23b2a98dad3e32bb1a2cd598f72ef7666026da71203e7577901
  Stored in directory: 

In [23]:
import requests
import os, sys
import urllib.request
import re
import subprocess
import shlex
import random
import deepspeech
import ffmpeg
import srt
import numpy as np

from datetime import datetime
from pytube import YouTube

# Downloading

In [3]:
def downloadBackground(URLs):
  for url in URLs:
    yt = YouTube(url)
    title = yt.title.replace(" ","-")
    if os.path.exists('/content/' + title + '.mp4'):
      print("Background " + title + " --> Already Downloaded")
    else:
      video_stream = yt.streams.filter(file_extension='mp4').get_highest_resolution()
      video_stream.download(filename=title + '.mp4')
      getMutedBackground('/content/' + title + '.mp4')   
      print("Background " + title + " --> Successfully Downloaded")

In [4]:
def downloadMP4FromURL(video_url,video_name):
  # Send an HTTP GET request to the URL
  response = requests.get(video_url)

  # Check the status code of the response
  if response.status_code == 200:
    # Write the content of the response to a file
    dir = '/content/' + video_name.split('.mp4')[0]
    os.makedirs(dir)
    with open(dir + '/' + video_name, 'wb') as f:
        f.write(response.content)
        print('Video Succesfully Downloaded')
  else:
    print('Failed to download video')


In [5]:
def getDOMTree(link_url,video_name):

  if os.path.exists('/content/' + video_name.split('.mp4')[0]):
    print('Video Already Downloaded')
  else:
    #HAVE TO MASK AS USER SO DOMAIN DOESN'T RECOGNIZE WEB SCRAPER
    HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
    r = requests.get(link_url,headers=HEADERS)
    index = r.text.find('contentUrl')

    if index != -1:
      # Get the line that contains the string by slicing the text
      line = r.text[index:].split('\n')[0]
      match = re.search(r'contentUrl": "(.+?)"', line)

      # Check if a match was found
      if match:
        # Get the URL from the match object
        url = match.group(1)
        downloadMP4FromURL(url,video_name)
      else:
        print('URL not found')
    else:
      print('String not found')

In [6]:
def fix_timestamp(timestamp):
  # Use a regular expression to check if the timestamp starts with '00:'
  if not re.match(r'^00:', timestamp):
    # If the timestamp does not start with '00:', add it to the beginning
    timestamp = '00:' + timestamp
  return timestamp + '.000'

# Cropping

In [16]:
def cropVideo(video_name, backgroundPath):

  outputPath = '/content/' + video_name.split('.mp4')[0]
  inputFile = outputPath + '/' + video_name

  startTime = input("Enter Start Timestamp: ")
  endTime = input("Enter End Timestamp: ")
  fileName = startTime + '-' + endTime + '.mp4'

  startTime = fix_timestamp(startTime)
  endTime = fix_timestamp(endTime)

  # 20 represents the quality/filesize ratio. 0 is best quality. 51 is worst quality.
  command = ['ffmpeg', '-i', inputFile, '-copyinkf', '-ss', startTime, '-to', endTime, '-vf', 'scale=1080:960,setsar=9/16', '-c:v', 'libx264', '-crf', '20', '-c:a', 'copy', f'{outputPath}/tateClip.mp4']
  subprocess.run(command)

  backgroundLength = getSeconds(getVideoLength(backgroundPath)[0])
  vidLength = getSeconds(getVideoLength(f'{outputPath}/tateClip.mp4')[0])
  
  backgroundStart = getRandomNumber(30, backgroundLength - vidLength - 30)

  command = ['ffmpeg', '-i', backgroundPath, '-copyinkf', '-ss', getHHMMSS(backgroundStart), '-to', getHHMMSS(backgroundStart + vidLength), '-vf', 'scale=1080:960,setsar=9/16', '-c:v', 'libx264', '-crf', '20', '-c:a', 'copy', '/content/background.mp4']
  subprocess.run(command)

  #ffmpeg -i top.mp4 -i bottom.mp4 -filter_complex "[0:v]scale=iw:ih/2[top];[1:v]scale=iw:ih/2[bottom];[top][bottom]vstack,setdar=9/16,format=yuv420p" -c:v libx264 -crf 20 output.mp4

  # Fix this
  command = [
      "ffmpeg",
      "-i",
      f'{outputPath}/tateClip.mp4',
      "-i",
      '/content/background.mp4',
      "-filter_complex",
      "[0:v]scale=iw:ih/2[top];[1:v]scale=iw:ih/2[bottom];[top][bottom]vstack,setdar=9/16,format=yuv420p",
      "-c:v",
      "libx264",
      "-crf",
      "20",
      f'{outputPath}/{fileName}'
  ]
  subprocess.run(command)

  # os.remove(f'{outputPath}/tateClip.mp4')
  os.remove('/content/background.mp4')

  return outputPath, f'{outputPath}/{fileName}'

In [8]:
def getVideoLength(video_path):
  command = "ffmpeg -i " + video_path + " 2>&1 | grep 'Duration' | cut -d ' ' -f 4 | sed s/,//"
  try:
    stdout = !{command}
    return stdout
  except:
    print("getVideoLength failed")
  # Get the duration of the video from the command output

In [9]:
def getMutedBackground(video_path):
  muted_video_path = '/content/muted.mp4'
  command = "ffmpeg -i " + video_path + " -c copy -an -y " + muted_video_path
  print(command)
  try:
    !{command}
    os.remove(video_path)
    os.rename(muted_video_path,video_path)  
  except:
    print("getMutedBackground failed")

In [10]:
def getSeconds(timestamp):
  dt = datetime.strptime(timestamp, "%H:%M:%S.%f")
  return dt.hour*3600 + dt.minute*60 + dt.second

In [11]:
def getHHMMSS(timestamp):
  dt = datetime.utcfromtimestamp(int(timestamp))
  return dt.strftime('%H:%M:%S') + '.000'

In [12]:
def getRandomNumber(a, b):
  # Get a random integer between a and b (inclusive)
  return random.randint(a, b)

# User Action Required

In [13]:
#@title Video Game Backgrounds

# Concatenate background Youtube links to this array
URLs = ['https://youtu.be/9NgBLirYQS8']
downloadBackground(URLs)

ffmpeg -i /content/Forza-Horizon-4-DRIVING-LIKE-A-BOSS!!-1876HP-RTR-Mustang.mp4 -c copy -an -y /content/muted.mp4
ffmpeg version 5.0 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/content/build --pkg-config-flags=--static --extra-cflags=-I/content/build/include --extra-ldflags=-L/content/build/lib --extra-libs='-lpthread -lm -lgomp' --ld=g++ --bindir=/content/build/bin --disable-ffplay --disable-ffprobe --disable-doc --disable-htmlpages --disable-manpages --disable-podpages --disable-txtpages --enable-version3 --enable-gpl --enable-nonfree --enable-opencl --enable-opengl --disable-shared --enable-static --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfdk-aac --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libkvazaar --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable

In [14]:
#@title URL Input
# https://odysee.com/@tatespeech:c/superbike-vs-supercar:1
link_url = input("Enter Odysee Link URL: ")
backgroundPath = '/content/Forza-Horizon-4-DRIVING-LIKE-A-BOSS!!-1876HP-RTR-Mustang.mp4'

Enter Odysee Link URL: https://odysee.com/@tatespeech:c/superbike-vs-supercar:1


In [25]:
#@title User Actions From URL Input
video_name = link_url.rsplit('/',1)[-1] + '.mp4'

# Downloads the correspondent Andrew Tate video
# Creates a repository, and stores .mp4 file
# If already downloaded, then skipped
getDOMTree(link_url,video_name)

# Crops both the background .mp4, and the Andrew Tate .mp4. Vertically concatenates them to be 9:16.
# Returns filepath of the video that contains the conjoined background .mp4 + Tate .mp4
clipDir, clipPath = cropVideo(video_name,backgroundPath)

# Create subtitles with AI
audioToSubtitles(clipDir)


Video Already Downloaded
Enter Start Timestamp: 02:20
Enter End Timestamp: 02:30


KeyboardInterrupt: ignored

# AI-Generated Subtitles

In [27]:
audioToSubtitles('/content/superbike-vs-supercar:1')

RuntimeError: ignored

In [26]:
def audioToSubtitles(clipDir):
  # Set the path to the DeepSpeech model files
  model_path = "/path/to/deepspeech-0.9.3-models"

  # Load the DeepSpeech model
  model = deepspeech.Model(model_path + "/output_graph.pbmm")

  # Set the beam width for the decoder
  beam_width = 500

  # Load the alphabet file
  alphabet_path = model_path + "/alphabet.txt"
  with open(alphabet_path, "r") as alphabet_file:
      alphabet = alphabet_file.read()

  # Set the language code
  language_code = "en-US"

  # Set the name of the input audio file
  audio_file = clipDir + '/tateClip.mp4'

  # Set the name of the output subtitle file
  subtitle_file = clipDir + '/subtitles.srt'

  # Extract the audio from the video file using FFmpeg
  audio_stream, _ = (
      ffmpeg
      .input(audio_file)
      .output("pipe:", format="s16le", acodec="pcm_s16le", ac=1, ar="16k")
      .run(capture_stdout=True)
  )

  # Convert the audio stream to a NumPy array
  audio = np.frombuffer(audio_stream, np.int16)

  # Transcribe the audio using DeepSpeech
  transcription = model.stt(audio, beam_width, language_code, alphabet)

  # Split the transcription into segments based on the timing of the audio
  segments = []

  # Set the duration of each segment (in seconds)
  segment_duration = 10

  # Set the start time of the first segment
  start_time = 0

  # Set the end time of the first segment
  end_time = start_time + segment_duration

  # Split the transcription into segments
  while start_time < len(audio) / 16000:
      # Get the text for the current segment
      segment_text = transcription[start_time:end_time]

      # Add the segment to the list of segments
      segments.append(segment_text)

      # Set the start time of the next segment
      start_time = end_time

      # Set the end time of the next segment
      end_time = start_time + segment_duration

  # Create a list of SRT entries
  entries = []

  # Set the start time of the first entry
  start_time = 0

  # Set the end time of the first entry
  end_time = segment_duration

  # Create SRT entries for each segment
  for segment in segments:
      # Create a new SRT entry
      entry = srt.Subtitle()

      # Set the start and end times of the entry
      entry.start = start_time
      entry.end = end_time

      # Set the text of the entry
      entry.content = segment

      # Add the entry to the list of entries
      entries.append(entry)

      # Set the start time of the next entry
      start_time = end_time

      # Set the end time of the next entry
      end_time = start_time + segment_duration

  # Save the entries to the output file
  with open(subtitle_file, "w") as subtitle_file:
      subtitle_file.write(srt.compose(entries))